# Задания по уроку 5.2

In [3]:
import pandas as pd
import numpy as np
import itertools

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier,
                              RandomTreesEmbedding, VotingClassifier)
from sklearn.linear_model import LogisticRegression


Все задания выполняются на основе датасета UCI ML Breast Cancer Wisconsin. (https://goo.gl/U2Uwz2)  
Все признаки являются числовыми.

In [4]:
X, y = load_breast_cancer(return_X_y=True)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

## Задачи с проверкой кода

1) Реализовать функцию нахождения самого частого элемента в массиве (голосование)

In [6]:
def get_most_frequent_value(sequence):
    return max(set(sequence), key = sequence.count)

In [7]:
get_most_frequent_value([1, 2, 3, 3, 3, 4, 4])

3

2) Реализовать функцию, выполняющую бутстреп.

In [12]:
def bootstrap(values, new_dataset_size):
    """
    Creates a new dataset from the old one using bootstrap.
    
    :arg values: np.array of shape (n_objects, n_features), input objects
    :arg new_dataset_size: int, number of elements in the resulting array
    
    :return: bootstraped_dataset: np.array of shape (new_dataset_size, n_features)
    """
    indices = np.random.randint(0, len(values), (new_dataset_size, len(data)))
    bootstraped_dataset = data[indices]
    return bootstraped_dataset

3) Реализовать фукнцию для подсчета произвольной метрики из Scikit-Learn в режиме OOB. Подсказка: `oob_decision_function_`

In [13]:
# Write-up
def estimate_oob_metric(forest, metric):
    """
    Computes any classification metric in the out-of-bag mode.
        :arg forest: Scikit-Learn ensemble model
        :arg metric: callable with two arguments that returns a float
        
        :return oob_metric_value: float
    """
    return forest.oob_score_ 

In [10]:
forest = RandomForestClassifier(n_estimators=100, max_depth=8, criterion='entropy', oob_score=True, random_state=42)
forest.fit(X_train, y_train)
y_pred = forest.predict(X_train)
f1 = f1_score(y_train, y_pred)

In [11]:
estimate_oob_metric(forest, f1_score)

0.967032967032967

## Задачи с проверкой ответа

В следующих заданих требуется так или иначе изменить параметры дерева решений и оценить результат.  
Как метрику качества будем использовать F1-score. В каждом задании необходимо поменять __только один__ параметр.  

Случайный лес — стохастический алгоритм, а значит при каждом запуске может выдавать различный результат. В дальнейших задачах __необходимо__ выставить случайность: добавить параметр `random_state=8` как аргумент при создании ансамблей.

In [14]:
def compute_metric(clf, X_train=X_train, y_train=y_train,
                   X_test=X_test, y_test=y_test):
    np.random.seed(42)
    clf.fit(X_train, y_train)
    y_test_pred = clf.predict(X_test)
    return np.round(f1_score(y_test, y_test_pred), 6)

Посчитаем значение метрики со стандартными параметрами.

In [15]:
forest = RandomForestClassifier(random_state=8)
compute_metric(forest)

0.965035

1) Напишите индекс самого важного признака по мнению случайного леса. Важности признаков указаны в переменной feature_importances_ случайного леса.

In [16]:
forest = RandomForestClassifier()
forest.fit(X_train, y_train)
print('Важность признаков:', forest.feature_importances_)
max_value = max(forest.feature_importances_)
print('Максимальное значение:', max_value)
indx = list(forest.feature_importances_).index(max_value)
print('Индекс:', indx)

Важность признаков: [0.04870337 0.01359088 0.05326975 0.04755501 0.00728533 0.01394433
 0.06800084 0.10620999 0.00377029 0.00388577 0.02013892 0.00472399
 0.01130301 0.02240696 0.00427091 0.00525322 0.00938583 0.00351326
 0.00401842 0.00532146 0.07798688 0.02174901 0.06711483 0.15389236
 0.01064421 0.02026604 0.0318016  0.14466327 0.01012018 0.00521012]
Максимальное значение: 0.15389236463205394
Индекс: 23


2) Увеличьте количество в случайном лесе до 100. Как изменилось качество? Укажите его.

In [17]:
forest = RandomForestClassifier(random_state=8, n_estimators=100)
compute_metric(forest)

0.965035

3) Выставите количество деревьев в лесе равным 100 и включите режим подсчета out-of-bag score. Укажите полученное значение метрики accuracy, округленного до 6 знака после запятой.

In [18]:
def calc_accuracy(y_true, y_pred):
    P = np.sum(y_true == 1)
    N = np.sum(y_true == 0)
    TP = np.sum((y_true == 1) & (y_pred == 1))
    TN = np.sum((y_true == 0) & (y_pred == 0))

    acc = (TP + TN) / (P + N)
    
    return acc, TP, TN, P, N

In [19]:
forest = RandomForestClassifier(random_state=8, n_estimators=100, oob_score=True)
forest.fit(X_train, y_train)
y_pred = forest.predict(X_train)
acc, TP, TN, P, N = calc_accuracy(y_train, y_pred)
print('TP = {}\nTN = {}\nP = {}\nN = {}\nacc = {}%'.format(TP, TN, P, N, acc * 100))

TP = 286
TN = 169
P = 286
N = 169
acc = 100.0%


4) Выключите режим random subspaces в случайном лесе. Укажите качество.

In [20]:
forest = RandomForestClassifier(n_estimators=100, oob_score=True)
compute_metric(forest)

0.972222

5) Выключите режим бутстрапа и переобучите случайный лес. Укажите качество.

In [21]:
forest = RandomForestClassifier(n_estimators=100, bootstrap=False)
compute_metric(forest)

0.965035

6) Какая средняя глубина у деревьев, которые получаются в случайном лесе со стандартными параметрами при его обучении на этой обучающей выборке?

In [22]:
forest = RandomForestClassifier()
forest.fit(X_train, y_train)
sum_depth = 0
for i in range(forest.n_estimators):
    sum_depth = sum_depth + forest.estimators_[0].tree_.max_depth

print('Средняя глубина =', sum_depth/forest.n_estimators)

Средняя глубина = 7.0


Помимо важностей, которые можно определить встроенным в Scikit-Learn способом, существует еще парочка других, основанных на модификации признаков. Некоторые из них находятся в библиотеке `rfpimp`:

In [17]:
! pip install rfpimp

7) При помощи библиотеки rfpimp определите индекс самого важного признака методом drop-column importance. Как классификатор используйте случайный лес со стандартными настройками.  
Важность необходимо оценить по __отложенной__ выборке.

8) При помощи библиотеки rfpimp определите индекс самого важного признака методом column-permutation importance. Как классификатор используйте случайный лес со стандартными настройками.  
Важность необходимо оценить по __подвыборке__ обучающей выборки (сэмплирование выполняется автоматически внутри библиотеки)

9) Мы познакомились со случайным лесом, который комбинирует бутстрап и random subspaces. Теперь познакомимся с методом, который добавляет еще один уровень случайности — экстремальные деревься (Extra Trees). В этом методе сплиты при построении деревьев строятся случайным образом. Замените RandomForestClassifier на ExtraTreesClassifier с такими же настройками. Укажите полученное качество ниже.

10) Познакомимся с Random Trees Embedding — преобразованием, которое отображает объекты в новое пространство признаков. Оно выполняется при помощи построения Extra Trees. Новое пространство признаков — пространство векторов, которые равны по длине количеству листьев деревьев всего леса. Для каждого объекта в этом векторе проставляется единица, если объект попал в лист при выполнении предсказания решающим деревом. Примените его для преобразования признаков, а затем поверх этих признаков обучите логистическую регрессию. Какое получилось качество? Измерьте его функцией `compute_metric`, выставив необходимые аргументы.

11) Объедините hard голосованием случайный лес, extra trees и логистическую регрессию. Для этого можно воспользоваться классом VotingClassifier. Укажите полученное качество.

12) Объедините soft голосованием случайный лес, extra trees и логистическую регрессию. Укажите полученное качество.

13) Объедините с весами `[0.8, 0.2]` soft голосованием extra trees и логистическую регрессию. Укажите полученное качество.